In [5]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [6]:
import mxnet.gluon.data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [7]:
for X, y in data_iter:
    print(X, y)
    break


[[-0.24494731 -0.81835526]
 [-0.37563023  1.0995659 ]
 [-0.70237225 -0.8823735 ]
 [ 0.7313352   0.8966576 ]
 [-1.2758839  -0.31069455]
 [-0.47648966 -0.0273269 ]
 [-1.4916571  -0.21315534]
 [-0.86072665 -0.3446788 ]
 [-0.5477563   0.42602822]
 [-0.53193796  1.1954637 ]]
<NDArray 10x2 @cpu(0)> 
[ 6.490005   -0.2905834   5.802512    2.61524     2.6947415   3.3317502
  1.91748     3.6619663   1.6484756  -0.93345016]
<NDArray 10 @cpu(0)>


In [8]:
from mxnet.gluon import nn

net = nn.Sequential()

In [9]:
net.add(nn.Dense(1))

In [12]:
print(net)

Sequential(
  (0): Dense(None -> 1, linear)
)


In [14]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()  # 平方损失又称L2范数损失
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

D:\ruanjian\conda\envs\gluon\lib\site-packages\mxnet\gluon\parameter.py:862: UserWarning: Parameter 'dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


In [15]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.054540
epoch 2, loss: 0.000246
epoch 3, loss: 0.000050
epoch 4, loss: 0.000049
epoch 5, loss: 0.000049
epoch 6, loss: 0.000049
epoch 7, loss: 0.000049
epoch 8, loss: 0.000049
epoch 9, loss: 0.000049
epoch 10, loss: 0.000049


In [17]:
dense = net[0]
print(true_w, dense.weight.data())
print(true_b, dense.bias.data())

[2, -3.4] 
[[ 1.9999956 -3.4003165]]
<NDArray 1x2 @cpu(0)>
4.2 
[4.2002406]
<NDArray 1 @cpu(0)>
